In [ ]:
from json import *
from copy import *
from heapq import *

#A state is a tuple of hallway, rooms, cost past
#The hallway is a list of chars of occupants or whitespaces for empty
#Rooms are an hallway index, a type of occupants and a list of the actual occupants
#Energy is the amount of energy to reach this state
#Past (if any) is a reference to the state that was unfolded to this

costs = {
    'A':1,
    'B':10,
    'C':100,
    'D':1000    
}

def heurestic(state):
    hallway, rooms, energy, past = state
    
    est = 0
    
    home = {}
    for i in range(len(rooms)):
        ix,tp,lst = rooms[i]
        home[tp]= rooms[i]

    #for every room 
    for ri in range(len(rooms)):
        ix,tp,lst = rooms[ri]

        #walkthrough list
        for li in range(len(lst)):            

            #if its a stranger
            if lst[li] != tp:
                #add base cost of travelling between rooms
                ix2,tp2,lst2 = home[lst[li]]
                est += (abs(ix - ix2) + 2) * costs[lst[li]]

    #scan hallway
    for i in range(len(hallway)):
        a = hallway[i]
        #find distance to every home + one step in 
        if a != ' ':
            ix,tp,lst = home[a]
            est += (abs(i - ix) + 1) * costs[a]

    
    #Todo add cost if someone is blocking
    #todo add cost of moving others in queue

    return est 

def room_has_strangers(lst, tp):
    for a in lst:
        if a != tp:
            return True
    return False

def isgoal(state):
    hallway, rooms, energy, past = state
    for x in hallway:
        if x != ' ':
            return False
    
    for ix,tp,lst in rooms:
        if room_has_strangers(lst, tp):
            return False
    return True


def state_parse(text):
    lines = text.split("\n")
    M = [list(l) for l in lines]
    
    hallway = []
    x=1
    y=1
    while M[y][x] == '.':
        hallway.append(' ')
        x+=1
    
    rooms = []
    for x in range(len(lines[2])):
        y=2
        r = []
        if M[y][x] != '#':
            r = []
            while M[y][x] != '#':
                r.append(M[y][x])
                y+=1
            rooms.append([x-1,chr(ord('A')+len(rooms)),r])
    
    energy = 0
    past = None
    return hallway, rooms, energy,  past


#print(state_parse(open("i23_test0.txt").read()))
#print(state_parse(open("i23_test1.txt").read()))
#print(state_parse(open("i23_test2.txt").read()))
#print(state_parse(open("i23_test3.txt").read()))
#print(state_parse(open("i23_test4.txt").read()))
#print(state_parse(open("i23_test5.txt").read()))
#print(state_parse(open("i23_test6.txt").read()))
#print(state_parse(open("i23.txt").read()))

def state_print(state):
    hallway, rooms, energy, past = state
    
    lines = []
    
    roomsize = 0
    for ix, tp, lst in rooms:
        roomsize = max(len(lst), roomsize)
        
    
    W = len(hallway) + 2
    
    lines.append("".join(["#" for x in range(W)]))
    lines.append("#"+"".join(hallway)+"#")
    
    for i in range(roomsize):
        l = ["#" for j in range(W)]
        for ix, tp, lst in rooms:
            if i < len(lst):
                l[ix+1] = lst[i]
            else:
                l[ix+1] = " "
        lines.append("".join(l))
    lines.append("".join(["#" for x in range(W)]))    
    for i in range(len(lines)):
        l = lines[i]
        f = rooms[0][0]
        if i > 2:
            l = " "*f + l[f:-f] 
            
        print(l)
    

#state_print(state_parse(open("i23_test0.txt").read()))
#state_print(state_parse(open("i23_test1.txt").read()))
#state_print(state_parse(open("i23_test2.txt").read()))
#state_print(state_parse(open("i23_test3.txt").read()))
#state_print(state_parse(open("i23_test4.txt").read()))
#state_print(state_parse(open("i23_test5.txt").read()))



def state_unfold(state):
    newstates = []
    hallway, rooms, energy, past = state
    #go through all move options and spawn new situations
    
    #pickup room info for hallway moves
    home = {}
    
    #build table of rooms
    for i in range(len(rooms)):
        ix, tp, lst = rooms[i]
        home[ix]=(i, rooms[i])
        
    #All moves in situation go from room to hallway or vice versa
    hr = range(len(hallway))
    for a in home.keys(): #room
        for b in hr:
            if b not in home.keys(): #hallway
                #verify path between indices are open
                dx = min(max(b-a,-1), 1)
                path_is_clear = True
                t = a + dx
                while t != b:
                    if hallway[t] != ' ':
                        path_is_clear = False
                        break
                    t+=dx

                if path_is_clear:
                    i, (ix, tp, lst) = home[a]                            
                    if room_has_strangers(lst, tp):
                        if hallway[b] == ' ':                                    
                            #room occupant can move to b in hallway
                            hallway2, rooms2, energy2, past = deepcopy(state)                                        
                            #energy from moving to door
                            energy2 += costs[lst[0]]*abs(a-b)
                            #energy from entering and moving others 
                            for tp in rooms2[i][2]:
                                energy2 += costs[tp]                                        
                            hallway2[b] = rooms2[i][2].pop(0)
                            newstates.append((hallway2, rooms2, energy2, state))
                    else:
                        if hallway[b] == tp:
                            #b in hallway can move directly to room
                            hallway2, rooms2, energy2, past = deepcopy(state)                                        
                            rooms2[i][2].insert(0, hallway2[b])
                            hallway2[b] = ' '
                            #energy from moving to door
                            energy2 += costs[tp]*abs(a-b)
                            #energy from entering and moving others 
                            for tp in rooms2[i][2]:
                                energy2 += costs[tp]                                        
                            newstates.append((hallway2, rooms2, energy2, state))
    
    #we are missing the direct room to room option, needed in example!!!
    
    
    return newstates

    
def state_playback(state):
    l = []
    while True:
        l.append(state)
        if state[-1] is None:
            break
        else:
            state = state[-1]
    
    l.reverse()
    for state in l:
        hallway, rooms, energy, past = state
        print("energy:%d"%(energy))
        state_print(state)
        print()

def state_hash(state):
    hallway, rooms, energy, past = state
    r = "".join(hallway)+","+",".join(["".join(lst) for ix,tp,lst in rooms])
    return r

    
def solve(fn, facit=None, playback=None):
    print("solving %s"%(fn))
    text = open(fn).read()
    #print(text)
    start_state = state_parse(text)
    #print(start_state)
    #state_print(start_state)
    
    evaluate = []
    evaluate.append((heurestic(start_state)+start_state[2], start_state))
    visited = {}
    i = 0
    
    last_heurestic = None
    printinterval = 1000
    
    while len(evaluate):
        
        srt, state = heappop(evaluate)
        
        hallway, rooms, energy, past = state
        
        if i % printinterval == 0 and i != 0:
            printinterval *= 2
            print("%s step:%d queue:%d visited:%d cost:%d heurestic:%d"%(fn, i, len(evaluate), len(visited.keys()), energy, heurestic(state)))
        
        if isgoal(state):
            conclusion = "goal!!! iterations:%d energy:%d"%(i, energy)
            if facit is None:
                pass
            else:
                if energy == facit:
                    conclusion += " OK :)"
                else:
                    conclusion += " ERROR!!! (should be %d)"%(facit)
            
            print(conclusion)
                    
            if playback:
                print("="*20)
                print("PLAYBACK!")
                state_playback(state)
                print("="*20)
                print()            
            return energy
        
        #note that we visited this state
        visited[state_hash(state)] = 1        
        
        #unfold state to new states
        newstates = state_unfold(state)
        
        for new_state in newstates:
            if not state_hash(new_state) in visited:
                heappush(evaluate, (heurestic(new_state)+new_state[2], new_state))        
        i+=1
        
    print("No solution found")
    return None
    
solve("i23_test0.txt",         46, False)
solve("i23_test1.txt",        112, False)
solve("i23_test2.txt",        810, False)
solve("i23_test3.txt",        719, False)
solve("i23_test4.txt",       8470, False)
solve("i23_test5.txt",      12521, False)
solve("i23.txt",            14510, False) #14510 - won me a gold star
solve("i23_part2_test.txt", 44169, False) #44775
solve("i23_part2.txt",      49180, False) #49180 - won me a gold star

print("fin")


solving i23_test0.txt
goal!!! iterations:6 energy:46 OK :)
solving i23_test1.txt
goal!!! iterations:136 energy:112 OK :)
solving i23_test2.txt
goal!!! iterations:33 energy:810 OK :)
solving i23_test3.txt
goal!!! iterations:510 energy:719 OK :)
solving i23_test4.txt
i23_test4.txt step:1000 queue:2236 visited:403 cost:7359 heurestic:2262
i23_test4.txt step:2000 queue:3174 visited:569 cost:7684 heurestic:2009
i23_test4.txt step:4000 queue:3616 visited:917 cost:3563 heurestic:6608
goal!!! iterations:7906 energy:10692 ERROR!!! (should be 8470)
solving i23_test5.txt
i23_test5.txt step:1000 queue:2534 visited:279 cost:1600 heurestic:6488
i23_test5.txt step:2000 queue:4417 visited:609 cost:3891 heurestic:4598
i23_test5.txt step:4000 queue:7765 visited:1172 cost:2154 heurestic:8478
i23_test5.txt step:8000 queue:12381 visited:1978 cost:3664 heurestic:8066
i23_test5.txt step:16000 queue:14708 visited:2670 cost:5695 heurestic:6099
i23_test5.txt step:32000 queue:19801 visited:5544 cost:5645 heurest

In [ ]:
def state_unfold(state):
    newstates = []
    hallway, rooms, energy, past = state
    #go through all move options and spawn new situations
    
    #pickup room info for hallway moves
    home = {}
    
    #build table of rooms
    for i in range(len(rooms)):
        ix, tp, lst = rooms[i]
        home[ix]=(i, rooms[i])
        
    #All moves in situation go from room to hallway or vice versa
    hr = range(len(hallway))
    for a in home.keys(): #room
        for b in hr:
            #verify path between indices are open
            dx = min(max(b-a,-1), 1)
            path_is_clear = True
            t = a + dx            
            while t != b:
                if hallway[t] != ' ':
                    path_is_clear = False
                    break
                t+=dx
                
            if path_is_clear:
                i, (ix, tp, lst) = home[a]
                if b in home.keys(): 
                    #room a to room b
                    if room_has_strangers(lst, tp):
                        i_b, (ix_b, tp_b, lst_b) = home[b]
                        if not room_has_strangers(lst_b, tp_b):
                    

                    
                else:
                    
                    if room_has_strangers(lst, tp):
                        #Room (a) to hallway (b)
                        if hallway[b] == ' ':                                    
                            #room occupant can move to b in hallway
                            hallway2, rooms2, energy2, past = deepcopy(state)                                        
                            #energy from moving to door
                            energy2 += costs[lst[0]]*abs(a-b)
                            #energy from entering and moving others 
                            for tp in rooms2[i][2]:
                                energy2 += costs[tp]                                        
                            hallway2[b] = rooms2[i][2].pop(0)
                            newstates.append((hallway2, rooms2, energy2, state))
                    else:
                        #Hallway to room
                        if hallway[b] == tp:
                            #b in hallway can move directly to room
                            hallway2, rooms2, energy2, past = deepcopy(state)                                        
                            rooms2[i][2].insert(0, hallway2[b])
                            hallway2[b] = ' '
                            #energy from moving to door
                            energy2 += costs[tp]*abs(a-b)
                            #energy from entering and moving others 
                            for tp in rooms2[i][2]:
                                energy2 += costs[tp]                                        
                            newstates.append((hallway2, rooms2, energy2, state))
    
    #we are missing the direct room to room option!!!
    
    
    return newstates
